# 区块链

关于Simchain中的区块链结构，示例如下：

In [1]:
from simchain import Network
net = Network()
# 5次随机交易、5次共识
for _ in range(5):
    net.make_random_transactions()
    net.consensus()
chain = net.peers[0].blockchain
# 获取区块的哈希值列表
[block.hash for block in chain]

2019-06-12 21:45:00,113 - A blockchain p2p network created,12 peers joined
2019-06-12 21:45:00,155 - genesis block has been generated
2019-06-12 21:45:00,198 - peer(24, 49)(pid=8) created a transaction
2019-06-12 21:45:00,220 - peer(24, 49)(pid=8) sent a transaction to network
2019-06-12 21:45:01,058 - peer(24, 49)(pid=8)'s transaction verified by 11 peers
2019-06-12 21:45:01,095 - peer(12, 69)(pid=5) created a transaction
2019-06-12 21:45:01,097 - peer(12, 69)(pid=5) sent a transaction to network
2019-06-12 21:45:01,925 - peer(12, 69)(pid=5)'s transaction verified by 11 peers
2019-06-12 21:45:01,967 - peer(14, 100)(pid=11) created a transaction
2019-06-12 21:45:01,971 - peer(14, 100)(pid=11) sent a transaction to network
2019-06-12 21:45:02,794 - peer(14, 100)(pid=11)'s transaction verified by 11 peers
2019-06-12 21:45:02,836 - peer(12, 69)(pid=5) created a transaction
2019-06-12 21:45:02,839 - peer(12, 69)(pid=5) sent a transaction to network
2019-06-12 21:45:03,681 - peer(12, 69)(pi

2019-06-12 21:46:35,964 - peer(85, 59)(pid=9) sent a transaction to network
2019-06-12 21:46:37,654 - peer(85, 59)(pid=9)'s transaction verified by 10 peers
2019-06-12 21:46:37,696 - peer(85, 59)(pid=9) created a transaction
2019-06-12 21:46:37,699 - peer(85, 59)(pid=9) sent a transaction to network
2019-06-12 21:46:38,534 - peer(85, 59)(pid=9)'s transaction verified by 11 peers
2019-06-12 21:46:38,536 - 8 peers are mining
2019-06-12 21:46:50,467 - peer(96, 38)(pid=2) is winner,11.93068242073059 secs used
2019-06-12 21:46:57,848 - Block(hash:00003dbc03c3c1e1b66f1406ec843bab7e3037ebf55409bb3436912943aa81be) received by 11 peers
2019-06-12 21:46:57,901 - peer(14, 100)(pid=11) created a transaction
2019-06-12 21:46:57,903 - peer(14, 100)(pid=11) sent a transaction to network
2019-06-12 21:46:58,773 - peer(14, 100)(pid=11)'s transaction verified by 11 peers
2019-06-12 21:46:58,812 - peer(16, 99)(pid=3) created a transaction
2019-06-12 21:46:58,814 - peer(16, 99)(pid=3) sent a transaction t

['de79b5f372e6c18a86124b90b750e76a1e32511be251514052bb09cef09e5276',
 '00000d818801b4287334a6000d6586923db2b25ee55f09d3138ad74d411dbf64',
 '00000272a558c6fd895567bf792dff872a0d2cd039537da7143c9fed6f3dd4b3',
 '0000184f31ae23cede54b9e60e70b0d8701984bb4812c6c414f3f84b7d8f53f4',
 '00003dbc03c3c1e1b66f1406ec843bab7e3037ebf55409bb3436912943aa81be',
 '000001c9005751cc1a4eac712afbb00c41ee593ff3657a06cee0478808006c32']

2019-06-12 21:47:25,864 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
# 获取前区块哈希值列表
[block.prev_block_hash for block in chain]

[None,
 'de79b5f372e6c18a86124b90b750e76a1e32511be251514052bb09cef09e5276',
 '00000d818801b4287334a6000d6586923db2b25ee55f09d3138ad74d411dbf64',
 '00000272a558c6fd895567bf792dff872a0d2cd039537da7143c9fed6f3dd4b3',
 '0000184f31ae23cede54b9e60e70b0d8701984bb4812c6c414f3f84b7d8f53f4',
 '00003dbc03c3c1e1b66f1406ec843bab7e3037ebf55409bb3436912943aa81be']

In [3]:
# 获取高度为3的区块
chain[2]

Block(hash:00000272a558c6fd895567bf792dff872a0d2cd039537da7143c9fed6f3dd4b3)

## 创世区块的创建过程

In [4]:
def create_genesis_block(self, number, value):
    self.init_peers(number=number)
    
    # 交易的输入列表
    tx_in = [Vin(to_spend=None,
                 signature=b'I love blockchain',
                 pubkey=None)]
    
    # 交易的输出列表，输出单元指向节点的地址
    tx_out = [Vout(value=value, to_addr=peer.wallet.addrs[-1])
              for peer in self.peers]

    # 创世区块中的交易
    txs = [Tx(tx_in=tx_in, tx_out=tx_out, nlocktime=0)]
    
    # 创世区块中的内容，时间戳是作者的生日
    # 难度位数可以在params模块中进行调整
    genesis_block = Block(version=0,
                          prev_block_hash=None,
                          timestamp=841124,
                          bits=0,
                          nonce=0,
                          txs=txs)

    logger.info('A blockchain p2p network created,{0} peers joined'.format(self.nop))
    logger.info('genesis block has been generated')

    utxos = find_utxos_from_block(txs)
    for peer in self.peers:
        peer.blockchain.append(genesis_block)
        add_utxos_to_set(peer.utxo_set, utxos)

当创世区块创建完成后，还需要将创世区块中的交易输出封装成UTXO添加到UTXO_SET。Simchain中定义了三种封装在节点添加区块到区块链时，UTXO的函数。

In [5]:
# 将交易列表中的所有输出单元封装到UTXO列表，默认为未确认
def find_utxos_from_txs(txs):
    return [UTXO(vout, Pointer(tx.id, i), tx.is_coinbase)
            for tx in txs for i, vout in enumerate(tx.tx_out)]

# 将交易列表中的所有输出单元封装到UTXO列表，默认为已确认
def find_utxos_from_block(txs):
    return [UTXO(vout, Pointer(tx.id, i), tx.is_coinbase, True, True)
            for tx in txs for i, vout in enumerate(tx.tx_out)]

# 将单笔交易的所有输出单元封装到UTXO列表，默认为未确认
def find_utxos_from_tx(tx):
    return [UTXO(vout, Pointer(tx.id, i), tx.is_coinbase)
            for i, vout in enumerate(tx.tx_out)]

当交易的所有输出单元被封装为UTXO后，则可以添加到UTXO_SET中，也定义了三个函数。

In [6]:
# 将单笔交易的输出单元封装成UTXO并添加到UTXO_SET中，未确认
def add_utxos_from_tx_to_set(utxo_set, tx):
    # 调用find_utxos_from_tx()函数封装UTXO
    utxos = find_utxos_from_tx(tx)
    # 将所有UTXO添加到UTXO_SET中
    for utxo in utxos:
        utxo_set[utxo.pointer] = utxo

# 将交易列表的输出单元封装成UTXO并添加到UTXO_SET，未确认
def add_utxo_from_txs_to_set(utxo_set, txs):
    # 调用find_utxos_from_txs()函数封装UTXO
    utxos = find_utxos_from_txs(txs)
    add_utxos_to_set(utxo_set, utxos)

# 将区块交易的输出单元封装成UTXO并添加值UTXO_SET，已确认
def add_utxo_from_block_to_set(utxo_set, txs):
    # 调用find_utxos_from_block()函数封装UTXO
    utxos = find_utxos_from_block(txs)
    add_utxos_to_set(utxo_set, utxos)

以上有了两个函数调用add_utxos_to_set()函数，该函数将UTXO列表或字典添加到UTXO_SET中

In [7]:
# 将UTXO列表或字典添加到UTXO_SET
def add_utxos_to_set(utxo_set, utxos):
    # 如果utxos是字典类型，则对其values进行操作
    if isinstance(utxos, dict):
        utxos = utxos.values()

    # 将所有UTXO添加到UTXO_SET中
    for utxo in utxos:
        utxo_set[utxo.pointer] = utxo

示例如下：

In [8]:
from simchain import Vin, Vout, Tx, Block, SigningKey
from simchain.ecc import convert_pubkey_to_addr
import random

In [9]:
# 创建5个整数作为私钥数值
ks = [random.randint(0, 100000) for _ in range(5)]
ks

[9013, 20872, 59153, 32192, 95719]

In [10]:
# 通过数值创建5个私钥对象
sks = [SigningKey.from_number(k) for k in ks]
# 获取对应的公钥和地址
pks = [sk.get_verifying_key() for sk in sks]
addrs = [convert_pubkey_to_addr(pk.to_bytes()) for pk in pks]

In [11]:
# 交易输入
tx_in = [Vin(to_spend=None, signature=None, pubkey=None)]
# 交易输出，每个地址1000
tx_out = [Vout(1000, addr) for addr in addrs]
# 创建创币交易
tx = Tx(tx_in, tx_out)
# 创建创世区块
gensis_block = Block(0, None, '0014', 18, 0, [tx])
# 初始化UTXO_SET
utxo_set = {}

In [12]:
from simchain.peer import find_utxos_from_tx, add_utxo_from_block_to_set
# 将单笔交易的输出单元封装成UTXO
utxos = find_utxos_from_tx(tx)
utxos

[UTXO(vout:Vout(to_addr:1000,value:1PCsag7595KT3rotfk4XRy267cCTk49kvo),pointer:Pointer(tx_id:1b936dffcc05529e60dfd03465a8ad162ebca7021b8d53ec1adc3278409dee3e,n:0)),
 UTXO(vout:Vout(to_addr:1000,value:1AkmdaL18yA4yvzhjLpczLtY6rg17DCzfk),pointer:Pointer(tx_id:1b936dffcc05529e60dfd03465a8ad162ebca7021b8d53ec1adc3278409dee3e,n:1)),
 UTXO(vout:Vout(to_addr:1000,value:151gYqzSzcemwvzhyXFeusWb16jfE2J4qW),pointer:Pointer(tx_id:1b936dffcc05529e60dfd03465a8ad162ebca7021b8d53ec1adc3278409dee3e,n:2)),
 UTXO(vout:Vout(to_addr:1000,value:1DJMiQGy2zQWSmdLaEfTk1r7gweKasUTBk),pointer:Pointer(tx_id:1b936dffcc05529e60dfd03465a8ad162ebca7021b8d53ec1adc3278409dee3e,n:3)),
 UTXO(vout:Vout(to_addr:1000,value:1Lm7HSGuvagZZhs7vN4yoWs4kLae7Nx3Qh),pointer:Pointer(tx_id:1b936dffcc05529e60dfd03465a8ad162ebca7021b8d53ec1adc3278409dee3e,n:4))]

In [13]:
# 5个UTXO，如果每个地址对应一个节点，则每个节点一个UTXO
len(utxos)

5

In [14]:
# 从单笔交易封装的UTXO添加到UTXO_SET
add_utxos_from_tx_to_set(utxo_set, tx)
list(utxo_set.values()) == utxos

True

In [15]:
# 所有UTXO未确认
[utxo.confirmed for utxo in utxos]

[False, False, False, False, False]

In [16]:
from simchain.peer import find_utxos_from_block, add_utxo_from_block_to_set
# 获取创世区块交易列表
txs = gensis_block.txs
# 从交易列表封装UTXO
utxos1 = find_utxos_from_block(txs)
utxo_set1 = {}
# 将区块交易输出封装成UTXO添加到UTXO_SET
add_utxo_from_block_to_set(utxo_set1, txs)
utxos1 == list(utxo_set1.values())

True

In [17]:
# 所有UTXO已确认
[utxo.confirmed for utxo in utxos1]

[True, True, True, True, True]